In [22]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / ".."))

import pickle
import torch

In [23]:
def pickle_load(filename: str |Path) -> torch.Tensor:
    with open(filename, "rb") as f:
        loaded = pickle.load(f)
    return loaded

In [29]:
root_path = list(Path.cwd().parents)[0]
gbi_data_path = root_path.joinpath("gbi_results", "benchmark").glob("*/xos/*.pkl")
for path in gbi_data_path:
    print(pickle_load(path).shape)

/home/robin/projects/lab_project/neuralgbi_diffusion/gbi_results/benchmark/uniform/xos/xo_specified_unknown.pkl
torch.Size([10, 1])
/home/robin/projects/lab_project/neuralgbi_diffusion/gbi_results/benchmark/uniform/xos/xo_misspecified_known.pkl
torch.Size([10, 1])
/home/robin/projects/lab_project/neuralgbi_diffusion/gbi_results/benchmark/uniform/xos/theta_gt_known.pkl
torch.Size([10, 1])
/home/robin/projects/lab_project/neuralgbi_diffusion/gbi_results/benchmark/uniform/xos/theta_gt_unknown.pkl
torch.Size([10, 1])
/home/robin/projects/lab_project/neuralgbi_diffusion/gbi_results/benchmark/uniform/xos/xo_misspecified_unknown.pkl
torch.Size([10, 1])
/home/robin/projects/lab_project/neuralgbi_diffusion/gbi_results/benchmark/uniform/xos/xo_specified_known.pkl
torch.Size([10, 1])
/home/robin/projects/lab_project/neuralgbi_diffusion/gbi_results/benchmark/linear_gaussian/xos/xo_specified_unknown.pkl
torch.Size([10, 10])
/home/robin/projects/lab_project/neuralgbi_diffusion/gbi_results/benchmark/

In [73]:
torch.load("../data/observed_data/linear_gaussian_10.pt", weights_only=True).values()
oberserved_data_dir = root_path /  "data" / "observed_data"
gbi_data_dir = root_path /  "data" / "gbi_data"
gbi_data_path = root_path.joinpath("gbi_results", "benchmark").glob("*/xos/xo*.pkl")
for path in gbi_data_path:
    dataset_name = str(list(path.parents)[0].parent).split("/")[-1]
    dataset = torch.load(oberserved_data_dir / (dataset_name + "_10.pt"), weights_only=True)                  
    gbi_x = pickle_load(path)
    # get gt theta
    theta = pickle_load(path.parent / ("theta_gt_" + path.stem.split("_")[-1] + ".pkl"))
    if dataset_name == "gaussian_mixture":
        theta = theta[:, None].repeat(1, gbi_x.shape[1], 1)
        theta = theta.T.flatten(start_dim=1).T
        gbi_x = gbi_x.T.flatten(start_dim=1).T
    dataset["_x"] = gbi_x
    dataset["_theta"] = theta
    
    save_name = f"{dataset_name}_{len(gbi_x)}.pt"
    save_dir = gbi_data_dir / path.stem[3:] / save_name
    torch.save(dataset, save_dir)